## LangGraph Open Deep Research - Supervisor-Researcher Architecture

In this notebook, we'll explore the **supervisor-researcher delegation architecture** for conducting deep research with LangGraph.

You can visit this repository to see the original application: [Open Deep Research](https://github.com/langchain-ai/open_deep_research)

Let's jump in!

## What We're Building

This implementation uses a **hierarchical delegation pattern** where:

1. **User Clarification** - Optionally asks clarifying questions to understand the research scope
2. **Research Brief Generation** - Transforms user messages into a structured research brief
3. **Supervisor** - A lead researcher that analyzes the brief and delegates research tasks
4. **Parallel Researchers** - Multiple sub-agents that conduct focused research simultaneously
5. **Research Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings are combined into a comprehensive report

![Architecture Diagram](https://i.imgur.com/Q8HEZn0.png)

This differs from a section-based approach by allowing dynamic task decomposition based on the research question, rather than predefined sections.

---

# 🤝 Breakout Room #1
## Deep Research Foundations

In this breakout room, we'll understand the architecture and components of the Open Deep Research system.

## Task 1: Dependencies

You'll need API keys for Anthropic (for the LLM) and Tavily (for web search). We'll configure the system to use Anthropic's Claude Sonnet 4 exclusively.

In [1]:
import os
import getpass

os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

## Task 2: State Definitions

The state structure is hierarchical with three levels:

### Agent State (Top Level)
Contains the overall conversation messages, research brief, accumulated notes, and final report.

### Supervisor State (Middle Level)
Manages the research supervisor's messages, research iterations, and coordinating parallel researchers.

### Researcher State (Bottom Level)
Each individual researcher has their own message history, tool call iterations, and research findings.

We also have structured outputs for tool calling:
- **ConductResearch** - Tool for supervisor to delegate research to a sub-agent
- **ResearchComplete** - Tool to signal research phase is done
- **ClarifyWithUser** - Structured output for asking clarifying questions
- **ResearchQuestion** - Structured output for the research brief

Let's import these from our library: [`open_deep_library/state.py`](open_deep_library/state.py)

In [2]:
# Import state definitions from the library
from open_deep_library.state import (
    # Main workflow states
    AgentState,           # Lines 65-72: Top-level agent state with messages, research_brief, notes, final_report
    AgentInputState,      # Lines 62-63: Input state is just messages
    
    # Supervisor states
    SupervisorState,      # Lines 74-81: Supervisor manages research delegation and iterations
    
    # Researcher states
    ResearcherState,      # Lines 83-90: Individual researcher with messages and tool iterations
    ResearcherOutputState, # Lines 92-96: Output from researcher (compressed research + raw notes)
    
    # Structured outputs for tool calling
    ConductResearch,      # Lines 15-19: Tool for delegating research to sub-agents
    ResearchComplete,     # Lines 21-22: Tool to signal research completion
    ClarifyWithUser,      # Lines 30-41: Structured output for user clarification
    ResearchQuestion,     # Lines 43-48: Structured output for research brief
)

## Task 3: Utility Functions and Tools

The system uses several key utilities:

### Search Tools
- **tavily_search** - Async web search with automatic summarization to stay within token limits
- Supports Anthropic native web search and Tavily API

### Reflection Tools
- **think_tool** - Allows researchers to reflect on their progress and plan next steps (ReAct pattern)

### Helper Utilities
- **get_all_tools** - Assembles the complete toolkit (search + MCP + reflection)
- **get_today_str** - Provides current date context for research
- Token limit handling utilities for graceful degradation

These are defined in [`open_deep_library/utils.py`](open_deep_library/utils.py)

In [3]:
# Import utility functions and tools from the library
from open_deep_library.utils import (
    # Search tool - Lines 43-136: Tavily search with automatic summarization
    tavily_search,
    
    # Reflection tool - Lines 219-244: Strategic thinking tool for ReAct pattern
    think_tool,
    
    # Tool assembly - Lines 569-597: Get all configured tools
    get_all_tools,
    
    # Date utility - Lines 872-879: Get formatted current date
    get_today_str,
    
    # Supporting utilities for error handling
    get_api_key_for_model,          # Lines 892-914: Get API keys from config or env
    is_token_limit_exceeded,         # Lines 665-701: Detect token limit errors
    get_model_token_limit,           # Lines 831-846: Look up model's token limit
    remove_up_to_last_ai_message,    # Lines 848-866: Truncate messages for retry
    anthropic_websearch_called,      # Lines 607-637: Detect Anthropic native search usage
    openai_websearch_called,         # Lines 639-658: Detect OpenAI native search usage
    get_notes_from_tool_calls,       # Lines 599-601: Extract notes from tool messages
)

## Task 4: Configuration System

The configuration system controls:

### Research Behavior
- **allow_clarification** - Whether to ask clarifying questions before research
- **max_concurrent_research_units** - How many parallel researchers can run (default: 5)
- **max_researcher_iterations** - How many times supervisor can delegate research (default: 6)
- **max_react_tool_calls** - Tool call limit per researcher (default: 10)

### Model Configuration
- **research_model** - Model for research and supervision (we'll use Anthropic)
- **compression_model** - Model for synthesizing findings
- **final_report_model** - Model for writing the final report
- **summarization_model** - Model for summarizing web search results

### Search Configuration
- **search_api** - Which search API to use (ANTHROPIC, TAVILY, or NONE)
- **max_content_length** - Character limit before summarization

Defined in [`open_deep_library/configuration.py`](open_deep_library/configuration.py)

In [4]:
# Import configuration from the library
from open_deep_library.configuration import (
    Configuration,    # Lines 38-247: Main configuration class with all settings
    SearchAPI,        # Lines 11-17: Enum for search API options (ANTHROPIC, TAVILY, NONE)
)

## Task 5: Prompt Templates

The system uses carefully engineered prompts for each phase:

### Phase 1: Clarification
**clarify_with_user_instructions** - Analyzes if the research scope is clear or needs clarification

### Phase 2: Research Brief
**transform_messages_into_research_topic_prompt** - Converts user messages into a detailed research brief

### Phase 3: Supervisor
**lead_researcher_prompt** - System prompt for the supervisor that manages delegation strategy

### Phase 4: Researcher
**research_system_prompt** - System prompt for individual researchers conducting focused research

### Phase 5: Compression
**compress_research_system_prompt** - Prompt for synthesizing research findings without losing information

### Phase 6: Final Report
**final_report_generation_prompt** - Comprehensive prompt for writing the final report

All prompts are defined in [`open_deep_library/prompts.py`](open_deep_library/prompts.py)

In [5]:
# Import prompt templates from the library
from open_deep_library.prompts import (
    clarify_with_user_instructions,                    # Lines 3-41: Ask clarifying questions
    transform_messages_into_research_topic_prompt,     # Lines 44-77: Generate research brief
    lead_researcher_prompt,                            # Lines 79-136: Supervisor system prompt
    research_system_prompt,                            # Lines 138-183: Researcher system prompt
    compress_research_system_prompt,                   # Lines 186-222: Research compression prompt
    final_report_generation_prompt,                    # Lines 228-308: Final report generation
)

## ❓ Question #1:

Explain the interrelationships between the three states (Agent, Supervisor, Researcher). Why don't we just make a single huge state?

##### Answer:
I would say that the Agent, Supervisor, and Researcher states work together like a team: the Agent holds the big picture, the Supervisor coordinates the work, and the Researchers do the detailed work. Splitting them up keeps things organized and easier to scale. One giant state would get messy fast and be harder to manage.

## ❓ Question #2:

What are the advantages and disadvantages of importing these components instead of including them in the notebook?

##### Answer:
I think importing components keeps the notebook clean and reusable, which is great for bigger projects. The downside is you have to jump between files to understand everything. Inline code is easier to read at first, but harder to maintain long-term.

## 🏗️ Activity #1: Explore the Prompts

Open `open_deep_library/prompts.py` and examine one of the prompt templates in detail.

**Requirements:**
1. Choose one prompt template (clarify, brief, supervisor, researcher, compression, or final report)
2. Explain what the prompt is designed to accomplish
3. Identify 2-3 key techniques used in the prompt (e.g., structured output, role definition, examples)
4. Suggest one improvement you might make to the prompt

**YOUR CODE HERE** - Write your analysis in a markdown cell below

1. I have chosen research_system_prompt template.

2. This prompt is designed to guide a research assistant to gather information efficiently using web tools while thinking carefully between searches. 
It helps the assistant stay focused, avoid unnecessary searching, and stop once there’s enough information to answer confidently. 

3. Key techniques used:
    Clear role definition: It clearly defines the assistant as a research assistant with a specific job.
    Structured instructions: Step-by-step guidance makes the research process predictable and organized.
    Hard limits and stopping rules: Tool call budgets and stop conditions prevent over-searching and wasted effort.

4. I would add a short example showing what a good use of think_tool looks like. This would help ensure consistent and high-quality reflection after each search.

---

# 🤝 Breakout Room #2
## Building & Running the Researcher

In this breakout room, we'll explore the node functions, build the graph, and run wellness research.

## Task 6: Node Functions - The Building Blocks

Now let's look at the node functions that make up our graph. We'll import them from the library and understand what each does.

### The Complete Research Workflow

The workflow consists of 8 key nodes organized into 3 subgraphs:

1. **Main Graph Nodes:**
   - `clarify_with_user` - Entry point that checks if clarification is needed
   - `write_research_brief` - Transforms user input into structured research brief
   - `final_report_generation` - Synthesizes all research into final report

2. **Supervisor Subgraph Nodes:**
   - `supervisor` - Lead researcher that plans and delegates
   - `supervisor_tools` - Executes supervisor's tool calls (delegation, reflection)

3. **Researcher Subgraph Nodes:**
   - `researcher` - Individual researcher conducting focused research
   - `researcher_tools` - Executes researcher's tool calls (search, reflection)
   - `compress_research` - Synthesizes researcher's findings

All nodes are defined in [`open_deep_library/deep_researcher.py`](open_deep_library/deep_researcher.py)

### Node 1: clarify_with_user

**Purpose:** Analyzes user messages and asks clarifying questions if the research scope is unclear.

**Key Steps:**
1. Check if clarification is enabled in configuration
2. Use structured output to analyze if clarification is needed
3. If needed, end with a clarifying question for the user
4. If not needed, proceed to research brief with verification message

**Implementation:** [`open_deep_library/deep_researcher.py` lines 60-115](open_deep_library/deep_researcher.py#L60-L115)

In [6]:
# Import the clarify_with_user node
from open_deep_library.deep_researcher import clarify_with_user

### Node 2: write_research_brief

**Purpose:** Transforms user messages into a structured research brief for the supervisor.

**Key Steps:**
1. Use structured output to generate detailed research brief from messages
2. Initialize supervisor with system prompt and research brief
3. Set up supervisor messages with proper context

**Why this matters:** A well-structured research brief helps the supervisor make better delegation decisions.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 118-175](open_deep_library/deep_researcher.py#L118-L175)

In [7]:
# Import the write_research_brief node
from open_deep_library.deep_researcher import write_research_brief

### Node 3: supervisor

**Purpose:** Lead research supervisor that plans research strategy and delegates to sub-researchers.

**Key Steps:**
1. Configure model with three tools:
   - `ConductResearch` - Delegate research to a sub-agent
   - `ResearchComplete` - Signal that research is done
   - `think_tool` - Strategic reflection before decisions
2. Generate response based on current context
3. Increment research iteration count
4. Proceed to tool execution

**Decision Making:** The supervisor uses `think_tool` to reflect before delegating research, ensuring thoughtful decomposition of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 178-223](open_deep_library/deep_researcher.py#L178-L223)

In [8]:
# Import the supervisor node (from supervisor subgraph)
from open_deep_library.deep_researcher import supervisor

### Node 4: supervisor_tools

**Purpose:** Executes the supervisor's tool calls, including strategic thinking and research delegation.

**Key Steps:**
1. Check exit conditions:
   - Exceeded maximum iterations
   - No tool calls made
   - `ResearchComplete` called
2. Process `think_tool` calls for strategic reflection
3. Execute `ConductResearch` calls in parallel:
   - Spawn researcher subgraphs for each delegation
   - Limit to `max_concurrent_research_units` (default: 5)
   - Gather all results asynchronously
4. Aggregate findings and return to supervisor

**Parallel Execution:** This is where the magic happens - multiple researchers work simultaneously on different aspects of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 225-349](open_deep_library/deep_researcher.py#L225-L349)

In [9]:
# Import the supervisor_tools node
from open_deep_library.deep_researcher import supervisor_tools

### Node 5: researcher

**Purpose:** Individual researcher that conducts focused research on a specific topic.

**Key Steps:**
1. Load all available tools (search, MCP, reflection)
2. Configure model with tools and researcher system prompt
3. Generate response with tool calls
4. Increment tool call iteration count

**ReAct Pattern:** Researchers use `think_tool` to reflect after each search, deciding whether to continue or provide their answer.

**Available Tools:**
- Search tools (Tavily or Anthropic native search)
- `think_tool` for strategic reflection
- `ResearchComplete` to signal completion
- MCP tools (if configured)

**Implementation:** [`open_deep_library/deep_researcher.py` lines 365-424](open_deep_library/deep_researcher.py#L365-L424)

In [10]:
# Import the researcher node (from researcher subgraph)
from open_deep_library.deep_researcher import researcher

### Node 6: researcher_tools

**Purpose:** Executes the researcher's tool calls, including searches and strategic reflection.

**Key Steps:**
1. Check early exit conditions (no tool calls, native search used)
2. Execute all tool calls in parallel:
   - Search tools fetch and summarize web content
   - `think_tool` records strategic reflections
   - MCP tools execute external integrations
3. Check late exit conditions:
   - Exceeded `max_react_tool_calls` (default: 10)
   - `ResearchComplete` called
4. Continue research loop or proceed to compression

**Error Handling:** Safely handles tool execution errors and continues with available results.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 435-509](open_deep_library/deep_researcher.py#L435-L509)

In [13]:
# Import the researcher_tools node
from open_deep_library.deep_researcher import researcher_tools

### Node 7: compress_research

**Purpose:** Compresses and synthesizes research findings into a concise, structured summary.

**Key Steps:**
1. Configure compression model
2. Add compression instruction to messages
3. Attempt compression with retry logic:
   - If token limit exceeded, remove older messages
   - Retry up to 3 times
4. Extract raw notes from tool and AI messages
5. Return compressed research and raw notes

**Why Compression?** Researchers may accumulate lots of tool outputs and reflections. Compression ensures:
- All important information is preserved
- Redundant information is deduplicated
- Content stays within token limits for the final report

**Token Limit Handling:** Gracefully handles token limit errors by progressively truncating messages.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 511-585](open_deep_library/deep_researcher.py#L511-L585)

In [14]:
# Import the compress_research node
from open_deep_library.deep_researcher import compress_research

### Node 8: final_report_generation

**Purpose:** Generates the final comprehensive research report from all collected findings.

**Key Steps:**
1. Extract all notes from completed research
2. Configure final report model
3. Attempt report generation with retry logic:
   - If token limit exceeded, truncate findings by 10%
   - Retry up to 3 times
4. Return final report or error message

**Token Limit Strategy:**
- First retry: Use model's token limit × 4 as character limit
- Subsequent retries: Reduce by 10% each time
- Graceful degradation with helpful error messages

**Report Quality:** The prompt guides the model to create well-structured reports with:
- Proper headings and sections
- Inline citations
- Comprehensive coverage of all findings
- Sources section at the end

**Implementation:** [`open_deep_library/deep_researcher.py` lines 607-697](open_deep_library/deep_researcher.py#L607-L697)

In [15]:
# Import the final_report_generation node
from open_deep_library.deep_researcher import final_report_generation

## Task 7: Graph Construction - Putting It All Together

The system is organized into three interconnected graphs:

### 1. Researcher Subgraph (Bottom Level)
Handles individual focused research on a specific topic:
```
START → researcher → researcher_tools → compress_research → END
               ↑            ↓
               └────────────┘ (loops until max iterations or ResearchComplete)
```

### 2. Supervisor Subgraph (Middle Level)
Manages research delegation and coordination:
```
START → supervisor → supervisor_tools → END
            ↑              ↓
            └──────────────┘ (loops until max iterations or ResearchComplete)
            
supervisor_tools spawns multiple researcher_subgraphs in parallel
```

### 3. Main Deep Researcher Graph (Top Level)
Orchestrates the complete research workflow:
```
START → clarify_with_user → write_research_brief → research_supervisor → final_report_generation → END
                 ↓                                       (supervisor_subgraph)
               (may end early if clarification needed)
```

Let's import the compiled graphs from the library.

In [16]:
# Import the pre-compiled graphs from the library
from open_deep_library.deep_researcher import (
    # Bottom level: Individual researcher workflow
    researcher_subgraph,    # Lines 588-605: researcher → researcher_tools → compress_research
    
    # Middle level: Supervisor coordination
    supervisor_subgraph,    # Lines 351-363: supervisor → supervisor_tools (spawns researchers)
    
    # Top level: Complete research workflow
    deep_researcher,        # Lines 699-719: Main graph with all phases
)

## Why This Architecture?

### Advantages of Supervisor-Researcher Delegation

1. **Dynamic Task Decomposition**
   - Unlike section-based approaches with predefined structure, the supervisor can break down research based on the actual question
   - Adapts to different types of research (comparisons, lists, deep dives, etc.)

2. **Parallel Execution**
   - Multiple researchers work simultaneously on different aspects
   - Much faster than sequential section processing
   - Configurable parallelism (1-20 concurrent researchers)

3. **ReAct Pattern for Quality**
   - Researchers use `think_tool` to reflect after each search
   - Prevents excessive searching and improves search quality
   - Natural stopping conditions based on information sufficiency

4. **Flexible Tool Integration**
   - Easy to add MCP tools for specialized research
   - Supports multiple search APIs (Anthropic, Tavily)
   - Each researcher can use different tool combinations

5. **Graceful Token Limit Handling**
   - Compression prevents token overflow
   - Progressive truncation in final report generation
   - Research can scale to arbitrary depths

### Trade-offs

- **Complexity:** More moving parts than section-based approach
- **Cost:** Parallel researchers use more tokens (but faster)
- **Unpredictability:** Research structure emerges dynamically

## Task 8: Running the Deep Researcher

Now let's see the system in action! We'll use it to research wellness strategies for improving sleep quality.

### Setup

We need to:
1. Set up the wellness research request
2. Configure the execution with Anthropic settings
3. Run the research workflow

In [17]:
# Set up the graph with Anthropic configuration
from IPython.display import Markdown, display
import uuid

# Note: deep_researcher is already compiled from the library
# For this demo, we'll use it directly without additional checkpointing
graph = deep_researcher

print("✓ Graph ready for execution")
print("  (Note: The graph is pre-compiled from the library)")

✓ Graph ready for execution
  (Note: The graph is pre-compiled from the library)


### Configuration for Anthropic

We'll configure the system to use:
- **Claude Sonnet 4** for all research, supervision, and report generation
- **Tavily** for web search (you can also use Anthropic's native search)
- **Moderate parallelism** (1 concurrent researcher for cost control)
- **Clarification enabled** (will ask if research scope is unclear)

In [18]:
# Configure for Anthropic with moderate settings
config = {
    "configurable": {
        # Model configuration - using Claude Sonnet 4 for everything
        "research_model": "anthropic:claude-sonnet-4-20250514",
        "research_model_max_tokens": 10000,
        
        "compression_model": "anthropic:claude-sonnet-4-20250514",
        "compression_model_max_tokens": 8192,
        
        "final_report_model": "anthropic:claude-sonnet-4-20250514",
        "final_report_model_max_tokens": 10000,
        
        "summarization_model": "anthropic:claude-sonnet-4-20250514",
        "summarization_model_max_tokens": 8192,
        
        # Research behavior
        "allow_clarification": True,
        "max_concurrent_research_units": 1,  # 1 parallel researcher
        "max_researcher_iterations": 2,      # Supervisor can delegate up to 2 times
        "max_react_tool_calls": 3,           # Each researcher can make up to 3 tool calls
        
        # Search configuration
        "search_api": "tavily",  # Using Tavily for web search
        "max_content_length": 50000,
        
        # Thread ID for this conversation
        "thread_id": str(uuid.uuid4())
    }
}

print("✓ Configuration ready")
print(f"  - Research Model: Claude Sonnet 4")
print(f"  - Max Concurrent Researchers: 1")
print(f"  - Max Iterations: 2")
print(f"  - Search API: Tavily")

✓ Configuration ready
  - Research Model: Claude Sonnet 4
  - Max Concurrent Researchers: 1
  - Max Iterations: 2
  - Search API: Tavily


### Execute the Wellness Research

Now let's run the research! We'll ask the system to research evidence-based strategies for improving sleep quality.

The workflow will:
1. **Clarify** - Check if the request is clear (may skip if obvious)
2. **Research Brief** - Transform our request into a structured brief
3. **Supervisor** - Plan research strategy and delegate to researchers
4. **Parallel Research** - Researchers gather information simultaneously
5. **Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings combined into comprehensive report

In [19]:
# Create our wellness research request
research_request = """
I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please research the best evidence-based strategies for improving sleep quality and create a comprehensive sleep improvement plan for me.
"""

# Execute the graph
async def run_research():
    """Run the research workflow and display results."""
    print("Starting research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": research_request}]},
        config,
        stream_mode="updates"
    ):
        # Display each step
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print(f"\nResearch Brief Generated:")
                    print(f"{node_output['research_brief'][:500]}...")
            
            elif node_name == "supervisor":
                print(f"\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                        print(f"Tool calls: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print(f"\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n" + "="*60)
                    print("FINAL REPORT GENERATED")
                    print("="*60 + "\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Research workflow completed!")
    print("="*60)

# Run the research
await run_research()

Starting research workflow...



APIConnectionError: Connection error.

## Task 9: Understanding the Output

Let's break down what happened:

### Phase 1: Clarification
The system checked if your request was clear. Since you provided specific details about your sleep issues, it likely proceeded without asking clarifying questions.

### Phase 2: Research Brief
Your request was transformed into a detailed research brief that guides the supervisor's delegation strategy.

### Phase 3: Supervisor Delegation
The supervisor analyzed the brief and decided how to break down the research:
- Used `think_tool` to plan strategy
- Called `ConductResearch` to delegate to researchers
- Each delegation specified a focused research topic (e.g., sleep hygiene, circadian rhythm, blue light effects)

### Phase 4: Parallel Research
Researchers worked on their assigned topics:
- Each researcher used web search tools to gather information
- Used `think_tool` to reflect after each search
- Decided when they had enough information
- Compressed their findings into clean summaries

### Phase 5: Final Report
All research findings were synthesized into a comprehensive sleep improvement plan with:
- Well-structured sections
- Evidence-based recommendations
- Practical action items
- Sources for further reading

## Task 10: Key Takeaways & Next Steps

### Architecture Benefits
1. **Dynamic Decomposition** - Research structure emerges from the question, not predefined
2. **Parallel Efficiency** - Multiple researchers work simultaneously
3. **ReAct Quality** - Strategic reflection improves search decisions
4. **Scalability** - Handles token limits gracefully through compression
5. **Flexibility** - Easy to add new tools and capabilities

### When to Use This Pattern
- **Complex research questions** that need multi-angle investigation
- **Comparison tasks** where parallel research on different topics is beneficial
- **Open-ended exploration** where structure should emerge dynamically
- **Time-sensitive research** where parallel execution speeds up results

### When to Use Section-Based Instead
- **Highly structured reports** with predefined format requirements
- **Template-based content** where sections are always the same
- **Sequential dependencies** where later sections depend on earlier ones
- **Budget constraints** where token efficiency is critical

### Extend the System
1. **Add MCP Tools** - Integrate specialized tools for your domain
2. **Custom Prompts** - Modify prompts for specific research types
3. **Different Models** - Try different Claude versions or mix models
4. **Persistence** - Use a real database for checkpointing instead of memory

### Learn More
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [Open Deep Research Repo](https://github.com/langchain-ai/open_deep_research)
- [Anthropic Claude Documentation](https://docs.anthropic.com/)
- [Tavily Search API](https://tavily.com/)

## ❓ Question #3:

What are the trade-offs of using parallel researchers vs. sequential research? When might you choose one approach over the other?

##### Answer:
I think that parallel researchers are faster and great for broad topics, but they can be harder to coordinate. Sequential research is slower but more controlled and focused. I would choose parallel for speed and coverage, sequential for depth and clarity.

## ❓ Question #4:

How would you adapt this deep research architecture for a production wellness application? What additional components would you need?

##### Answer:
For a wellness app, I would adapt this by adding user profiles, safety checks, and personalization logic. I would also want data storage, monitoring, and feedback loops. This helps keep the advice relevant, safe, and improving over time.

## 🏗️ Activity #2: Custom Wellness Research

Using what you've learned, run a custom wellness research task.

**Requirements:**
1. Create a wellness-related research question (exercise, nutrition, stress, etc.)
2. Modify the configuration for your use case
3. Run the research and analyze the output
4. Document what worked well and what could be improved

**Experiment ideas:**
- Research exercise routines for specific conditions (bad knee, lower back pain)
- Compare different stress management techniques
- Investigate nutrition strategies for specific goals
- Explore meditation and mindfulness research

**YOUR CODE HERE**

In [20]:
# YOUR CODE HERE

# Configuration for custom wellness research (conservative demo setup)
import uuid

custom_config = {
    "configurable": {
        "research_model": "anthropic:claude-sonnet-4-20250514",
        "research_model_max_tokens": 6000,
        "compression_model": "anthropic:claude-sonnet-4-20250514",
        "compression_model_max_tokens": 5000,
        "final_report_model": "anthropic:claude-sonnet-4-20250514",
        "final_report_model_max_tokens": 7000,
        "summarization_model": "anthropic:claude-sonnet-4-20250514",
        "summarization_model_max_tokens": 5000,
        "allow_clarification": True,
        "max_concurrent_research_units": 1,
        "max_researcher_iterations": 1,
        "max_react_tool_calls": 2,
        "search_api": "tavily",
        "max_content_length": 40000,
        "thread_id": str(uuid.uuid4())
    }
}

print("✓ Configuration ready")
print(f"  - Research Model: Claude Sonnet 4")
print(f"  - Max Concurrent Researchers: 1")
print(f"  - Max Iterations: 1")
print(f"  - Search API: Tavily")


✓ Configuration ready
  - Research Model: Claude Sonnet 4
  - Max Concurrent Researchers: 1
  - Max Iterations: 1
  - Search API: Tavily


In [22]:
from IPython.display import Markdown, display

# My custom wellness research request
research_request = """
Create a research-backed 6-week cardiovascular conditioning plan for a 35-year-old beginner cyclist, currently riding 20-25 minutes at a comfortable pace, 
experiencing occasional lower back tightness — prioritizing spinal comfort, gradual workload increases, and long-term consistency.

Context:
Cycles 4 times per week
Does light yoga and mobility work at home
Goal is to build endurance and confidence without aggravating back discomfort
Please include:

Week-by-week cycling structure
Intensity guidance (easy spins vs steady efforts vs short challenges)
Complementary mobility and core-strength recommendations
Injury prevention tips (lower back, hips, hamstrings)
Safety considerations and signs to reduce or stop training
Sources / references
"""

# Execute the graph
async def run_research():
    """Run the custom wellness research workflow and display results."""
    print("Starting custom wellness research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": research_request}]},
        custom_config,
        stream_mode="updates"
    ):
        # Display progress for each node
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\nClarification step output:")
                    print(last_msg.content)
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print("\nResearch brief generated:")
                    print(node_output["research_brief"][:500] + "...")
            
            elif node_name == "supervisor":
                print("\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
                        print(f"Tool calls planned: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print("\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n{'='*60}")
                    print("FINAL REPORT GENERATED")
                    print(f"{'='*60}\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Custom wellness research workflow completed!")
    print("="*60)

# Run the research
await run_research()

Starting custom wellness research workflow...


Node: clarify_with_user

Clarification step output:
I have sufficient information to create your research-backed 6-week cardiovascular conditioning plan. I understand you need a beginner-friendly cycling program for a 35-year-old currently riding 20-25 minutes comfortably, 4 times per week, with occasional lower back tightness. The plan will focus on gradual progression, spinal comfort, complementary mobility/core work, and injury prevention strategies. I'll now begin researching evidence-based approaches for cardiovascular conditioning, cycling progression protocols, and lower back health considerations to develop your comprehensive 6-week program.

Node: write_research_brief

Research brief generated:
I need a comprehensive, research-backed 6-week cardiovascular conditioning plan specifically designed for a 35-year-old beginner cyclist who currently rides 20-25 minutes at a comfortable pace, 4 times per week, and experiences occasional 

# Research-Backed 6-Week Cardiovascular Conditioning Plan for Beginner Cyclists with Lower Back Considerations

## Week-by-Week Cycling Structure

### Week 1-2: Foundation Phase
**Frequency:** 4 rides per week
**Duration:** 
- 3 rides at 25-30 minutes
- 1 ride at 20 minutes (recovery ride)
**Intensity Distribution:** 80% easy pace, 20% moderate effort
**Progression:** Increase duration by 5 minutes from baseline while maintaining comfortable intensity

Research indicates that beginner cyclists benefit from an initial adaptation period where volume increases gradually while intensity remains low to moderate. The polarized training model, which emphasizes high volumes of low-intensity training (Zone 1-2), has been shown to be particularly effective for developing aerobic capacity in recreational cyclists.

### Week 3-4: Development Phase
**Frequency:** 4 rides per week
**Duration:**
- 2 rides at 35-40 minutes
- 1 ride at 30 minutes with structured intervals
- 1 ride at 25 minutes (recovery ride)
**Intensity Distribution:** 75% easy pace, 20% moderate effort, 5% higher intensity
**Progression:** Introduction of brief tempo efforts (2-3 minutes) within longer rides

Studies on cardiovascular conditioning demonstrate that incorporating moderate-intensity intervals after 2-3 weeks of base training optimizes aerobic adaptation while minimizing injury risk. The gradual introduction of structured efforts allows for physiological adaptation without overwhelming the musculoskeletal system.

### Week 5-6: Consolidation Phase
**Frequency:** 4 rides per week
**Duration:**
- 2 rides at 40-45 minutes
- 1 ride at 35 minutes with intervals
- 1 ride at 30 minutes (recovery ride)
**Intensity Distribution:** 70% easy pace, 25% moderate effort, 5% higher intensity
**Progression:** Longer sustained efforts (5-8 minutes) and one longer endurance ride

Research on training periodization shows that consolidation phases help solidify fitness gains while preparing the body for future training loads. The emphasis remains on aerobic development with controlled exposure to higher intensities.

## Intensity Guidance and Heart Rate Zones

### Heart Rate Zone Calculations
For a 35-year-old cyclist, using the Karvonen method with an estimated maximum heart rate of 185 bpm and assumed resting heart rate of 60 bpm:

**Zone 1 (Recovery/Easy Spin):** 50-60% HRR = 122-135 bpm
**Zone 2 (Steady Endurance):** 60-70% HRR = 135-147 bpm
**Zone 3 (Tempo/Moderate):** 70-80% HRR = 147-160 bpm
**Zone 4 (Threshold):** 80-90% HRR = 160-172 bpm

### Perceived Exertion Guidelines
Using the 1-10 RPE scale:
- **Easy Spins (Zone 1-2):** RPE 3-5, conversational pace, nasal breathing possible
- **Steady Efforts (Zone 2-3):** RPE 5-6, can speak in sentences, rhythmic breathing
- **Short Challenges (Zone 3-4):** RPE 6-7, can speak few words, controlled deeper breathing

Research demonstrates that RPE-based training is particularly effective for beginners who may not have access to heart rate monitors, with strong correlations between perceived exertion and physiological markers of intensity.

## Complementary Mobility and Core Strength Recommendations

### Pre-Ride Dynamic Mobility (5-8 minutes)
**Hip Circles:** 10 each direction
- Forward and backward leg swings: 10 each leg
- Thoracic spine rotations: 8 each direction
- Cat-cow stretches: 10 repetitions

Research on cycling-specific warm-up protocols shows that dynamic mobility exercises targeting the hip complex and thoracic spine significantly improve cycling performance and reduce lower back discomfort.

### Post-Ride Static Stretching (8-10 minutes)
**Hip Flexor Stretch:** 30 seconds each side
- Seated forward fold: 45 seconds
- Seated spinal twist: 30 seconds each side
- Child's pose with side reach: 30 seconds each side
- Supine knee-to-chest: 30 seconds each leg

Studies demonstrate that post-exercise static stretching, particularly targeting hip flexors and spinal extensors, can reduce cycling-related lower back tension and improve recovery.

### Core Strengthening Program (3x per week, alternating with cycling days)
**Week 1-2:**
- Dead bug: 2 sets of 8 each side
- Modified plank: 2 sets of 20-30 seconds
- Bird dog: 2 sets of 8 each side
- Glute bridge: 2 sets of 12

**Week 3-4:**
- Dead bug: 2 sets of 10 each side
- Plank: 2 sets of 30-45 seconds
- Bird dog: 2 sets of 10 each side
- Single-leg glute bridge: 2 sets of 8 each side
- Side plank (modified): 2 sets of 15-20 seconds each side

**Week 5-6:**
- Dead bug with resistance: 2 sets of 10 each side
- Plank: 2 sets of 45-60 seconds
- Bird dog with holds: 2 sets of 8 each side (3-second holds)
- Single-leg glute bridge: 2 sets of 10 each side
- Side plank: 2 sets of 20-30 seconds each side
- Pallof press (with resistance band): 2 sets of 8 each side

Research on core stability training for cyclists demonstrates that exercises targeting deep stabilizing muscles, particularly the transverse abdominis and multifidus, significantly reduce lower back pain and improve cycling efficiency.

## Comprehensive Injury Prevention Strategies

### Lower Back Prevention
**Bike Fit Considerations:**
- Saddle height should allow 25-30 degree knee bend at bottom of pedal stroke
- Saddle setback positioned so knee tracks over pedal axle when cranks are horizontal
- Handlebar height should not create excessive forward lean (>45 degrees initially)

Research on cycling biomechanics shows that proper bike fit is the primary factor in preventing lower back pain, with saddle height and fore-aft position being critical parameters.

**Posture Cues During Riding:**
- Maintain neutral spine position with slight forward lean
- Engage core muscles gently throughout ride
- Relax shoulders and avoid excessive grip tension
- Change hand positions every 10-15 minutes

### Hip and Hamstring Prevention
**Specific Interventions:**
- Daily hip flexor stretching (couch stretch or low lunge)
- Hamstring mobility work 3x per week (standing forward fold, seated stretches)
- Glute activation exercises before rides
- Focus on full pedal stroke engagement

Studies indicate that cycling's repetitive hip flexion can lead to adaptive shortening of hip flexors and hamstrings, making targeted mobility work essential for injury prevention.

### Progressive Loading Principles
- Increase weekly training volume by no more than 10%
- Follow hard training days with easy or rest days
- Include complete rest day every 7-10 days
- Monitor morning heart rate variability as recovery indicator

Research on training load management demonstrates that adherence to progressive loading principles reduces injury risk by up to 50% in recreational cyclists.

## Safety Considerations and Warning Signs

### Immediate Stop Indicators
**Sharp, sudden pain in:**
- Lower back (especially with radiation into legs)
- Knee (anterior, medial, or posterior)
- Neck or shoulders (indicating potential bike fit issues)

**Cardiovascular Warning Signs:**
- Chest pain or pressure
- Dizziness or lightheadedness
- Irregular heartbeat
- Excessive shortness of breath that doesn't resolve with reduced intensity

### Reduce Training Indicators
**Persistent muscle soreness lasting >48 hours**
- Elevated resting heart rate (>10 bpm above normal for 2+ consecutive days)
- General fatigue affecting daily activities
- Sleep disturbances
- Decreased motivation or enjoyment

Research on overtraining syndrome shows that early recognition of these warning signs and appropriate training modification can prevent more serious complications.

### Recovery and Modification Guidelines
**When experiencing minor lower back stiffness:**
- Reduce ride duration by 25-30%
- Focus on easier intensities (Zone 1-2 only)
- Increase pre and post-ride mobility work
- Consider bike fit assessment if symptoms persist >1 week

**When experiencing general fatigue:**
- Take an additional rest day
- Reduce intensity while maintaining duration
- Ensure adequate sleep (7-9 hours) and nutrition
- Consider stress management techniques

## Equipment and Environmental Considerations

### Essential Equipment for Back Health
**Proper cycling shorts with chamois**
- Well-fitted helmet to prevent neck strain
- Multiple hand position options (drop bars or ergonomic grips)
- Consider professional bike fit after 4-6 weeks of consistent riding

### Environmental Factors
**Weather considerations:**
- Cold weather requires extended warm-up (10-15 minutes)
- Hot weather necessitates increased hydration and intensity reduction
- Wind conditions may require route modification to maintain target intensities

Research demonstrates that environmental factors significantly impact exercise tolerance and injury risk, particularly for recreational cyclists who may be less accustomed to varying conditions.

## Monitoring and Progression Beyond 6 Weeks

### Success Metrics
**Objective Measures:**
- Ability to maintain conversation during Zone 2 efforts
- Completion of 45-minute rides without significant discomfort
- Improved recovery between training sessions
- Stable or improved morning heart rate

**Subjective Measures:**
- Increased confidence and enjoyment during rides
- Reduced lower back stiffness post-ride
- Enhanced energy levels throughout the day
- Motivation to continue cycling

### Long-term Progression Guidelines
After completing this 6-week program, research suggests that recreational cyclists can safely:
- Increase longest ride by 10-15 minutes every 2 weeks
- Gradually introduce longer interval sessions (up to 15-20 minutes)
- Consider participating in organized group rides or events
- Explore different cycling disciplines (mountain biking, touring, etc.)

The foundation established through this structured approach creates the physiological and biomechanical adaptations necessary for long-term cycling enjoyment and injury prevention.

### Sources

[1] Laursen, P.B., & Buchheit, M. (2019). Science and application of high-intensity interval training. Human Kinetics.

[2] Seiler, S. (2010). What is best practice for training intensity and duration distribution in endurance athletes? International Journal of Sports Physiology and Performance, 5(3), 276-291.

[3] Burnett, A.F., et al. (2004). Spinal kinematics and trunk muscle activity in cycling: implications for low back pain. Clinical Biomechanics, 19(4), 325-333.

[4] Salai, M., et al. (1999). Effect of changing the saddle angle on the incidence of low back pain in recreational bicyclists. British Journal of Sports Medicine, 33(6), 398-400.

[5] Van Hoof, W., et al. (2012). Degenerative changes of the spine in cycling: a systematic review. Sports Medicine, 42(6), 533-546.

[6] Akuthota, V., & Nadler, S.F. (2004). Core strengthening. Archives of Physical Medicine and Rehabilitation, 85(3), 86-92.

[7] Richardson, C.A., et al. (2002). Therapeutic exercise for spinal segmental stabilization in low back pain. Churchill Livingstone.

[8] Gabbett, T.J. (2016). The training-injury prevention paradox: should athletes be training smarter and harder? British Journal of Sports Medicine, 50(5), 273-280.

[9] Borg, G. (1998). Borg's perceived exertion and pain scales. Human Kinetics.

[10] McGill, S.M. (2007). Low back disorders: evidence-based prevention and rehabilitation. Human Kinetics.


Custom wellness research workflow completed!


What worked well:
The system turned a general wellness request into a clear and useful training plan with simple weekly structure, safety guidance, and trusted sources. Even with limited tools, the information was organized well and easy to follow.

What could be improved:
The plan could be better if it asked a few more questions about the user first. It would also help to focus sources more tightly and add a simple way to track progress week to week.